In [ ]:
import numpy as np
from module import *
import pypoman
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, convex_hull_plot_2d

In [ ]:
from plotting import generate_polygon, make_rectangle

In [ ]:
# import os
# figure_path = ''
# filename = os.path.join(figure_path, 'figure_3_b.pdf')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
theta_x = 0.4130544060912823
theta_y =  0.7202015331287421
theta_z_00 = 0.2159316891369294
theta_z_01 = 0.9386294150829768
theta_z_10 = 0.06753251255922954
theta_z_11 = 0.16402954207426068

# Get the vectors alpha, beta with the marginal distributions P_{XZ} and P_{YZ}
alpha = get_alpha(theta_x, theta_y, theta_z_00, theta_z_01, theta_z_10, theta_z_11)
beta = get_beta(theta_x, theta_y, theta_z_00, theta_z_01, theta_z_10, theta_z_11)

# Get "trivial" marginal bounds---lambda_x_min, lambda_x_max, lambda_y_min, lambda_y_max
lambda_x_min, lambda_x_max, a_0 = get_trivial_bounds(alpha)
lambda_y_min, lambda_y_max, b_0 = get_trivial_bounds(beta)

# Get the constraint matrices from the marginal probabilities
A, B = get_constraint_matrices(theta_x, theta_y)

# Get the bounds from structural causal marginal problem through linear program
LB_X, UB_X, LB_Y, UB_Y = get_linear_prog_bounds(a_0, b_0, A, B, lambda_x_min, lambda_x_max, lambda_y_min, lambda_y_max)


proj = compute_affine_projection(A, B, a_0, b_0)
eq = compute_equality_params(A, B, a_0, b_0)
ineq = compute_inequality_params(A, B, lambda_y_min, lambda_y_max, b_0, lambda_x_min, lambda_x_max, a_0)

# List of vertices of the projection of the polytope in R^{16}
vertices_projected_polytope = pypoman.projection.project_polytope(proj=proj, 
                                                                  ineq=ineq, 
                                                                      eq=eq)
points = np.vstack(vertices_projected_polytope)

trivial_bounds = make_rectangle(lambda_x_min, lambda_x_max, lambda_y_min, lambda_y_max)
lin_prog_bounds = make_rectangle(LB_X, UB_X, LB_Y, UB_Y)

area_trivial = area_polygon_points(trivial_bounds)
area_lin_prog = area_polygon_points(lin_prog_bounds)
area_polygon = area_polygon_points(points)

print('Polygon/Red: ',area_polygon/area_trivial)
print('Blue/Red: ',area_lin_prog/area_trivial)

edgecolor='black'
plt.figure(figsize=(4.5,4))

poly_trivial_bounds = generate_polygon(trivial_bounds, color='#e8cccc', edgecolor='#ca7f80', alpha=1.0, hatch='||')
plt.gca().add_patch(poly_trivial_bounds)

poly_lin_prog = generate_polygon(lin_prog_bounds, color='white', alpha=1.0)
plt.gca().add_patch(poly_lin_prog)
poly_lin_prog = generate_polygon(lin_prog_bounds, color='#cce2f8', edgecolor='grey', alpha=1.0, hatch='')
plt.gca().add_patch(poly_lin_prog)


poly = generate_polygon(points, color='white', alpha=1.0)
plt.gca().add_patch(poly)
poly = generate_polygon(points, color='#d3ffd3', edgecolor='#81c9c9', alpha=1.0, hatch='..')
plt.gca().add_patch(poly)

# x and y axes titles
xlabel = '$\lambda_X$'
ylabel = '$\lambda_Y$'
plt.xlabel(xlabel, fontsize=15, labelpad=5)
plt.ylabel(ylabel, fontsize=15, labelpad=5,rotation=0)
plt.xticks([lambda_x_min, LB_X, UB_X], ['$\lambda_X^{min}$', '$LB_X$', '$\lambda_X^{max} = UB_X$'])
plt.yticks([lambda_y_min, LB_Y, UB_Y], ['$\lambda_Y^{min}$', '$LB_Y$', '$\lambda_Y^{max} = UB_Y$'])

plt.ylim(-0.01 + lambda_y_min, lambda_y_max+0.01)
plt.xlim(-0.01 + lambda_x_min, lambda_x_max+0.01)
# plt.savefig(filename, dpi=None, facecolor='w', edgecolor='w',
#         orientation='portrait', format=None,
#         transparent=True, bbox_inches='tight', pad_inches=0.1, metadata=None)

plt.show()